In [1]:
import requests
from bs4 import BeautifulSoup
import re
from collections import Counter, defaultdict
from spacy.lang.xx import MultiLanguage
import pymorphy2
import pandas as pd
import numpy as np
morph = pymorphy2.MorphAnalyzer(lang="uk")
nlp = MultiLanguage()
tokenizer = MultiLanguage().Defaults.create_tokenizer(nlp)
from sklearn.metrics import (roc_auc_score, precision_score, recall_score, 
                             confusion_matrix, accuracy_score, f1_score)
# SITE = "https://rozetka.com.ua"
reg = re.compile("page=(\d+)/")
ADVANTANGE_TXT = "Переваги:\xa0"
DRAWBACK_TXT = "Недоліки:\xa0"
SCRAPE_FILE = "scrape_feedbacks.csv"

def get_data(url, counter, cls):
    url = url.format(counter)
    response = requests.get(url)
    if response.status_code != 200:
        print('END: request status:', response.status_code)
        return


    page = reg.search(response.url)
    if page:
        curr_page = int(page.groups()[0])
    else:
        curr_page = 1
    if curr_page != counter:
        print('END: final page reached:', counter-1)
        return
    soup = BeautifulSoup(response.text, 'html5lib')
    for script in soup(["script", "style"]):
        script.decompose()  # rip it out
    data = soup.select(cls)
    if not data:
        print('END: no data:')
        return
    return data

Get links to all mobile phones

In [2]:
items_links = []
page_c = 1
while True:
    data = get_data("https://rozetka.com.ua/ua/mobile-phones/c80003/page={}", page_c, '.g-i-tile-i-title a')
    if not data:
        break
    items_links += [i.attrs["href"] for i in data]
    page_c += 1
# items_links = ['https://rozetka.com.ua/ua/samsung_galaxy_s8_64gb_black/p15222497/', 'https://rozetka.com.ua/ua/huawei_p_smart_blue/p30270231/', 'https://rozetka.com.ua/ua/lenovo_pa530156ua/p12089470/', 'https://rozetka.com.ua/ua/xiaomi_redmi_note4_332_blk_not_www/p16937714/', 'https://rozetka.com.ua/ua/samsung_sm_j730fzsnsek/p18100440/', 'https://rozetka.com.ua/ua/samsung_galaxy_a8_2018_32gb_orchid_gray/p28487337/', 'https://rozetka.com.ua/ua/zte_nubia_m2_64gb_bl_gd_ru/p31032639/', 'https://rozetka.com.ua/ua/samsung_sm_j710fzkddek/p22469198/', 'https://rozetka.com.ua/ua/samsung_sm_a520fzkdsek/p13252176/', 'https://rozetka.com.ua/ua/nokia_8_ds_tempered_blue/p21275430/', 'https://rozetka.com.ua/ua/xiaomi_redmi_4x_332_gld_www/p16810070/', 'https://rozetka.com.ua/ua/xiaomi_redmi_5_plus_3_32gb_blue/p34965191/', 'https://rozetka.com.ua/ua/sony_xperia_xz1_compact_black/p24470978/', 'https://rozetka.com.ua/ua/xiaomi_mi_a1_4_32g_black/p29681111/', 'https://rozetka.com.ua/ua/samsung_sm_n950fzvdsek/p21617366/', 'https://rozetka.com.ua/ua/samsung_sm_j320hzkdsek/p7646250/', 'https://rozetka.com.ua/ua/prestigio_psp5552duogold/p31514791/', 'https://rozetka.com.ua/ua/huawei_y6_pro_wh/p7239508/', 'https://rozetka.com.ua/ua/samsung_sm_a720fzddsek/p13252316/', 'https://rozetka.com.ua/ua/huawei_mate_10_lite_4_64gb_blue/p25675161/', 'https://rozetka.com.ua/ua/lg_lgm700an_acisbk/p24252977/', 'https://rozetka.com.ua/ua/huawei_p8_lite_2017_black/p13721402/', 'https://rozetka.com.ua/ua/samsung_sm_j530fzknsek/p17891640/', 'https://rozetka.com.ua/ua/samsung_sm_j330fzkdsek/p19398505/', 'https://rozetka.com.ua/ua/xiaomi_redmi_note4x_3_32_blk_eu/p17489972/', 'https://rozetka.com.ua/ua/samsung_sm_j710fzdusek/p9141672/', 'https://rozetka.com.ua/ua/zte_blade_v8_gray/p25738433/', 'https://rozetka.com.ua/ua/samsung_galaxy_j1_2016_sm_j120h_gold/p7070984/', 'https://rozetka.com.ua/ua/meizu_m5_note_32gb_grey_euromart/p15300099/', 'https://rozetka.com.ua/ua/samsung_galaxy_a8_plus_2018_4_32gb_black/p28487353/', 'https://rozetka.com.ua/ua/lenovo_a1010a20_black/p11245840/', 'https://rozetka.com.ua/ua/samsung_galaxy_s9_plus_64gb_gray/p33767231/', 'https://rozetka.com.ua/ua/samsung_galaxy_s9_64gb_gray/p33767119/', 'https://rozetka.com.ua/ua/tplink_neffos_tp903a46ua/p25379089/', 'https://rozetka.com.ua/ua/tplink_neffos_tp902a24ua_gr/p14857859/', 'https://rozetka.com.ua/ua/tplink_neffos_tp903a2ua_gr/p30567479/', 'https://rozetka.com.ua/ua/impression_imsmart_a554/p18132432/', 'https://rozetka.com.ua/ua/tplink_neffos_tp904a44ua_gd/p23049211/', 'https://rozetka.com.ua/ua/xiaomi_redmi_note4_332_gry_not_www/p17474264/', 'https://rozetka.com.ua/ua/impression_imsmart_c502/p18132480/', 'https://rozetka.com.ua/ua/impression_imsmart_a504/p18132294/', 'https://rozetka.com.ua/ua/tp_link_neffos_c7_tp910a24ua/p32644807/', 'https://rozetka.com.ua/ua/xiaomi_redmi_note_5a_gray_4_64gb/p28299001/', 'https://rozetka.com.ua/ua/xiaomi_redmi_5a_2_16gb_grey/p30271319/', 'https://rozetka.com.ua/ua/alcatel_one_touch_idol_4s_6070k_dark_grey/p12863683/', 'https://rozetka.com.ua/ua/lenovo_pa570065ua/p12094384/', 'https://rozetka.com.ua/ua/leeco_s3_x622_rose_gold/p28185201/', 'https://rozetka.com.ua/ua/huawei_mate_10_pro_6_128gb_brown/p26927233/', 'https://rozetka.com.ua/ua/xiaomi_redmi_4a_2_16_gd_eu/p12544443/', 'https://rozetka.com.ua/ua/meizu_m6_note_3_16gb_gold_euromart/p31032511/', 'https://rozetka.com.ua/ua/nokia_150_ds_black/p13697280/', 'https://rozetka.com.ua/ua/xiaomi_redmi_4x_16_gold/p14936750/', 'https://rozetka.com.ua/ua/tp_link_neffos_c5a_tp703a21ua/p32642919/', 'https://rozetka.com.ua/ua/meizu_m6_216_black_eu/p31068007/', 'https://rozetka.com.ua/ua/doogee_y6_max_gold/p18816994/', 'https://rozetka.com.ua/ua/doogee_6924351601805/p12379075/', 'https://rozetka.com.ua/ua/xiaomi_redmi_5_3_32_black/p30394135/', 'https://rozetka.com.ua/ua/nokia_2_ds_white/p30860863/', 'https://rozetka.com.ua/ua/doogee_6924351602505/p14534570/', 'https://rozetka.com.ua/ua/meizu_m6_332_black_eu/p31072399/', 'https://rozetka.com.ua/ua/assistant_as_5436_grid_black/p25165665/', 'https://rozetka.com.ua/ua/meizu_m5_note_16gb_grey_euromart/p14519090/', 'https://rozetka.com.ua/ua/nokia_105_ds_ta_1034_black/p21853497/', 'https://rozetka.com.ua/ua/huawei_p20_lite_4_64gb_blue/p36378080/', 'https://rozetka.com.ua/ua/doogge_x5_max_pro_white/p10490260/', 'https://rozetka.com.ua/ua/meizu_m5_32_black/p20352500/', 'https://rozetka.com.ua/ua/leeco_cool_1_3_32gb_gold/p30233447/', 'https://rozetka.com.ua/ua/sigma_xtreme_pq24_black_orange/p13903808/', 'https://rozetka.com.ua/ua/lg_lgh930ds_acisbl/p31442159/', 'https://rozetka.com.ua/ua/sigma_xtreme_pq28_black_orange/p30946399/', 'https://rozetka.com.ua/ua/acer_hm_hp7eu_001/p3930505/', 'https://rozetka.com.ua/ua/meizu_m5c_2_16_black_eu/p18969811/', 'https://rozetka.com.ua/ua/alcatel_5025d_metallic_silver/p4751053/', 'https://rozetka.com.ua/ua/sigma_4827798283318/p25165537/', 'https://rozetka.com.ua/ua/nokia_216_dual_sim_grey/p11684285/', 'https://rozetka.com.ua/ua/leeco_le_s3_x552_silver/p30264143/', 'https://rozetka.com.ua/ua/impression_imsmart_c551/p18132510/', 'https://rozetka.com.ua/ua/sigma_x_treme_pq51_black_red/p32887143/', 'https://rozetka.com.ua/ua/sony_xperia_x_dual_f5122_rose_gold/p24251962/', 'https://rozetka.com.ua/ua/ergo_f501_magic_ds_black/p26392617/', 'https://rozetka.com.ua/ua/nokia_230_dual_sim_white/p6740302/', 'https://rozetka.com.ua/ua/xiaomi_redmi_note_5a_gold_3_32gb/p28288281/', 'https://rozetka.com.ua/ua/meizu_m5_216_blk_eu_www/p16937636/', 'https://rozetka.com.ua/ua/asus_zenfone_4_max_zc554kl_4g110ww/p25785961/', 'https://rozetka.com.ua/ua/impression_imsmart_a404/p18132174/', 'https://rozetka.com.ua/ua/doogee_s30_black/p24857232/', 'https://rozetka.com.ua/ua/impression_imsmart_c571/p18132564/', 'https://rozetka.com.ua/ua/xiaomi_redmi_4x_332_pink_euro/p26969553/', 'https://rozetka.com.ua/ua/prestigio_psp3423duoblack/p21196939/', 'https://rozetka.com.ua/ua/asus_zenfone4_ze554kl_1a009ww/p28564281/', 'https://rozetka.com.ua/ua/xiaomi_mi_a1_4_64g_gold/p25374649/', 'https://rozetka.com.ua/ua/leeco_le_pro_3_al_edition_x651_4_32gb_rose_gold/p30262487/', 'https://rozetka.com.ua/ua/leeco_le_pro_3_x650_4_64gb_gold/p28276785/', 'https://rozetka.com.ua/ua/huawei_nova_lite_2017_black/p22328125/', 'https://rozetka.com.ua/ua/huawei_51091rvg/p18145380/', 'https://rozetka.com.ua/ua/fly_ff244_grey/p22833016/', 'https://rozetka.com.ua/ua/ergo_f181_step/p25107273/', 'https://rozetka.com.ua/ua/lg_lgm230_acisbn/p15283628/', 'https://rozetka.com.ua/ua/bravis_c240_middle/p25040185/', 'https://rozetka.com.ua/ua/blackview_bv4000_pro_black/p28914241/', 'https://rozetka.com.ua/ua/blackview_a7_gold/p28409729/', 'https://rozetka.com.ua/ua/asus_zenfone_4_max_90ax00h1_m02180/p33892231/', 'https://rozetka.com.ua/ua/sony_xperia_xz1_black/p24478776/', 'https://rozetka.com.ua/ua/alcatel_5010d_volcano_black/p12235532/', 'https://rozetka.com.ua/ua/prestigio_psp3512duogold/p18201492/', 'https://rozetka.com.ua/ua/bravis_c180_jingle/p25039209/', 'https://rozetka.com.ua/ua/xiaomi_redmi_5_2_16gb_gold/p34788615/', 'https://rozetka.com.ua/ua/assistant_as_503_target_dark_blue/p32105991/', 'https://rozetka.com.ua/ua/astro_a172_black_red/p21682326/', 'https://rozetka.com.ua/ua/asus_zenfone_max_plus_m1_90ax0181_m00270/p33906951/', 'https://rozetka.com.ua/ua/nokia_105_ta_1010_blue/p21857725/', 'https://rozetka.com.ua/ua/nomi_i282_grey/p22334992/', 'https://rozetka.com.ua/ua/blackview_bv7000_grey_roz/p28376153/', 'https://rozetka.com.ua/ua/philips_xenium_e168_dual_sim_black/p27925473/', 'https://rozetka.com.ua/ua/lg_lgm700an_a4isbk/p27147665/', 'https://rozetka.com.ua/ua/ergo_f243_swift/p25051241/', 'https://rozetka.com.ua/ua/xiaomi_redmi_note_5a_gold_eu/p24229338/', 'https://rozetka.com.ua/ua/leeco_le_s3_x626_4_32gb_gld_int/p22685981/', 'https://rozetka.com.ua/ua/sigma_mobile_x_treme_it_67_dual_sim_black/p5853513/', 'https://rozetka.com.ua/ua/nomi_i5050_evo_z_dark_blue/p21777393/', 'https://rozetka.com.ua/ua/nokia_130_ds_ta_1017_black/p21856507/', 'https://rozetka.com.ua/ua/nous_ns_5005_fabulous_black/p29744415/', 'https://rozetka.com.ua/ua/nomi_i5013_evo_m2_pro_blue/p24226489/', 'https://rozetka.com.ua/ua/meizu_m5s_3_16_gold_eu/p18964085/', 'https://rozetka.com.ua/ua/caterpillar_cat_s60/p11034356/', 'https://rozetka.com.ua/ua/asus_zenfone_live_zb501kl_black/p23631534/', 'https://rozetka.com.ua/ua/sigma_mobile_comfort_50_mini3_bg/p2042292/', 'https://rozetka.com.ua/ua/huawei_p10_432_bk/p14609348/', 'https://rozetka.com.ua/ua/doogee_x20_1_16gb_silver_roz/p28306201/', 'https://rozetka.com.ua/ua/sigma_xtreme_pq68_black_yellow/p14430866/', 'https://rozetka.com.ua/ua/sony_xperia_xa1_plus_dual_blue/p24478818/', 'https://rozetka.com.ua/ua/xiaomi_redmi_5a_2_16gb_pink_roz/p31061239/', 'https://rozetka.com.ua/ua/sigma_mobile_x_treme_dz67_travel_black_black/p710774/', 'https://rozetka.com.ua/ua/sigma_comfort_50_shell_duo_blue/p16944452/', 'https://rozetka.com.ua/ua/sigma_mobile_comfort_elegance_50_red/p17619857/', 'https://rozetka.com.ua/ua/astro_b200_rx_black_yellow/p9235048/', 'https://rozetka.com.ua/ua/caterpillar_cat_s41/p32506247/', 'https://rozetka.com.ua/ua/bluboo_s8_3_32gb_black_6970595010323/p25469697/', 'https://rozetka.com.ua/ua/alcatel_5045d_2calua1/p24471328/', 'https://rozetka.com.ua/ua/nokia_3_ds_silver/p14653556/', 'https://rozetka.com.ua/ua/nokia_3310_2017_ds_yellow/p14652296/', 'https://rozetka.com.ua/ua/xiaomi_redmi_note_5a_gold/p24898854/', 'https://rozetka.com.ua/ua/samsung_sm_j510hzwdsek/p9141889/', 'https://rozetka.com.ua/ua/astro_a177_red/p9961900/', 'https://rozetka.com.ua/ua/nokia_5_ds_copper/p16104626/', 'https://rozetka.com.ua/ua/htc_desire_830_ds_wht_eu/p34221791/', 'https://rozetka.com.ua/ua/asus_ze551ml_delux_32gb_wht_eu/p18517856/', 'https://rozetka.com.ua/ua/asus_zenfone_4_max_zc554kl_4i111ww/p28523641/', 'https://rozetka.com.ua/ua/huawei_p10_plus_464_bk/p15275872/', 'https://rozetka.com.ua/ua/xiaomi_redmi_note_5a_prime_3_32gb_gray_roz/p30548327/', 'https://rozetka.com.ua/ua/fly_ff281_black/p10091428/', 'https://rozetka.com.ua/ua/lenovo_zuk_z1_3_64gb_wh_www_eu/p17918694/', 'https://rozetka.com.ua/ua/prestigio_pfp1240duoblack/p9244344/', 'https://rozetka.com.ua/ua/prestigio_psp5520duogold/p27248905/', 'https://rozetka.com.ua/ua/nous_ns_5006_rose_gold/p12438386/', 'https://rozetka.com.ua/ua/meizu_m5c_2_32_black_eu/p30434383/', 'https://rozetka.com.ua/ua/sony_xperia_xa1_ultra_dual_gold/p14658296/', 'https://rozetka.com.ua/ua/doogge_6924351603410/p12679529/', 'https://rozetka.com.ua/ua/lg_x_power_k220ds_gold/p11032858/', 'https://rozetka.com.ua/ua/nomi_i6030_note_x_black/p24227294/', 'https://rozetka.com.ua/ua/fly_fs408_stratus_8_black/p25222425/', 'https://rozetka.com.ua/ua/sigma_xtreme_pq14_black/p30254263/', 'https://rozetka.com.ua/ua/doogee_shoot_2_gold/p18817680/', 'https://rozetka.com.ua/ua/fly_ff180_4610015097566/p27749961/', 'https://rozetka.com.ua/ua/viaan_v1820_black/p24520909/', 'https://rozetka.com.ua/ua/prestigio_psp3510duored/p22126581/', 'https://rozetka.com.ua/ua/alcatel_one_touch_2007d_dark_grey/p5477928/', 'https://rozetka.com.ua/ua/nomi_i5532_space_x2_black/p24221176/', 'https://rozetka.com.ua/ua/nomi_i5510_space_m_black/p22343420/', 'https://rozetka.com.ua/ua/philips_xenium_e181_dual_sim_black/p12434613/', 'https://rozetka.com.ua/ua/alcatel_5080x_satin_volcano_black/p12982942/', 'https://rozetka.com.ua/ua/sigma_mobile_xtreme_it68_black/p4897731/', 'https://rozetka.com.ua/ua/nomi_i5012_evo_m2_blue/p24222422/', 'https://rozetka.com.ua/ua/lg_lgm700_aciskg/p20086297/', 'https://rozetka.com.ua/ua/sigma_xtreme_pq17_black_yellow/p23422500/', 'https://rozetka.com.ua/ua/blackview_a7_pro_gold/p28902449/', 'https://rozetka.com.ua/ua/prestigio_n3pfp1281duoblack/p31748607/', 'https://rozetka.com.ua/ua/huawei_p10_premium_464_bl/p19386556/', 'https://rozetka.com.ua/ua/htc_one_a9s_3_32gb_cast_iron/p28490753/', 'https://rozetka.com.ua/ua/lenovo_pa5e0028ua/p12094132/', 'https://rozetka.com.ua/ua/fly_ff282_black/p22838567/', 'https://rozetka.com.ua/ua/sigma_xtreme_io68_bobber/p21988569/', 'https://rozetka.com.ua/ua/assistant_as_5412_max_dark_blue/p19628630/', 'https://rozetka.com.ua/ua/asus_zenfone_4_max_zc554kl_4g060ww/p33159375/', 'https://rozetka.com.ua/ua/36389776/p36389776/', 'https://rozetka.com.ua/ua/36389800/p36389800/', 'https://rozetka.com.ua/ua/fly_ff243/p11022414/', 'https://rozetka.com.ua/ua/astro_a170_black_red/p21682319/', 'https://rozetka.com.ua/ua/alcatel_5044d_2calua1/p24470110/', 'https://rozetka.com.ua/ua/nomi_i246_red_338339/p34520927/', 'https://rozetka.com.ua/ua/asus_zc553kl_4h033ww/p18200040/', 'https://rozetka.com.ua/ua/twoe_r180_ds_black/p24869384/', 'https://rozetka.com.ua/ua/sigma_mobile_x_treme_3sim_black/p10223679/', 'https://rozetka.com.ua/ua/sigma_mobile_x_style_31_power_red/p32283735/', 'https://rozetka.com.ua/ua/philips_s327_dual_sim_blue/p27894401/', 'https://rozetka.com.ua/ua/viaan_v403_white/p34540487/', 'https://rozetka.com.ua/ua/prestigio_psp3468duoblack/p17834064/', 'https://rozetka.com.ua/ua/philips_xenium_e560_ds_black/p12656065/', 'https://rozetka.com.ua/ua/fly_ezzy_9_black/p22797246/', 'https://rozetka.com.ua/ua/bravis_a504_trace_gold/p13122718/', 'https://rozetka.com.ua/ua/astro_a284_red/p14429210/', 'https://rozetka.com.ua/ua/bravis_a506_crystal_gold/p18538856/', 'https://rozetka.com.ua/ua/motorola_pa90007ua/p21865600/', 'https://rozetka.com.ua/ua/samsung_sm_j105hzkdsek/p7622212/', 'https://rozetka.com.ua/ua/caterpillar_cat_s30/p5529297/', 'https://rozetka.com.ua/ua/huawei_y5_2017_51050nfd/p25246401/', 'https://rozetka.com.ua/ua/bravis_a553_discovey_gold/p18561319/', 'https://rozetka.com.ua/ua/apple_iphone_se_32gb_gold/p16853822/', 'https://rozetka.com.ua/ua/prestigio_pfp1180duwhite/p15215448/', 'https://rozetka.com.ua/ua/apple_iphone_x_64gb_silver/p22726294/', 'https://rozetka.com.ua/ua/nous_ns_5004_gold/p12437245/', 'https://rozetka.com.ua/ua/nomi_i5001_evo_m3_blue/p33483599/', 'https://rozetka.com.ua/ua/htc_u_play_eea_brill_blck/p15245464/', 'https://rozetka.com.ua/ua/xiaomi_redmi_5_3_32gb_gold/p34788839/', 'https://rozetka.com.ua/ua/prestigio_pfp1241duowhite/p9797568/', 'https://rozetka.com.ua/ua/apple_iphone_6_32gb_space_gray/p17217371/', 'https://rozetka.com.ua/ua/fly_ts113_black/p23049463/', 'https://rozetka.com.ua/ua/xiaomi_redmi_note4x_3_16_gld_eu/p17927640/', 'https://rozetka.com.ua/ua/caterpillar_cat_s31/p32512255/', 'https://rozetka.com.ua/ua/nomi_i241_plus_gold_319745/p34511303/', 'https://rozetka.com.ua/ua/lg_lgx240_acisku/p15215462/', 'https://rozetka.com.ua/ua/zte_axon7_mini_grey/p11891223/', 'https://rozetka.com.ua/ua/fly_fs458_stratus_7_black/p24502996/', 'https://rozetka.com.ua/ua/assistant_as_5421_gold/p28991129/', 'https://rozetka.com.ua/ua/sigma_x_treme_pq52_black_orange/p32905767/', 'https://rozetka.com.ua/ua/sigma_mobile_x_treme_3gsm_black_orange/p13743725/', 'https://rozetka.com.ua/ua/sigma_mobile_comfort_slim_50_b/p651114/', 'https://rozetka.com.ua/ua/apple_iphone_7_32gb_gold/p11256403/', 'https://rozetka.com.ua/ua/assistant_as_5435_black/p17927322/', 'https://rozetka.com.ua/ua/motorola_pa610107ua/p16808462/', 'https://rozetka.com.ua/ua/nomi_i281_red/p21812694/', 'https://rozetka.com.ua/ua/lg_x_power_2_lgm320_acisgd/p18878104/', 'https://rozetka.com.ua/ua/lephone_k10/p32523279/', 'https://rozetka.com.ua/ua/motorola_sm4482ajk7/p21850438/', 'https://rozetka.com.ua/ua/viaan_v11_black/p19882982/', 'https://rozetka.com.ua/ua/doogee_x30_gold_roz/p28288625/', 'https://rozetka.com.ua/ua/sony_xperia_l1_290742/p15285049/', 'https://rozetka.com.ua/ua/motorola_pa750032ua/p18852778/', 'https://rozetka.com.ua/ua/astro_s450_rx_bk_orng/p7655511/', 'https://rozetka.com.ua/ua/apple_iphone_7_plus_32gb_jet_black/p24385984/', 'https://rozetka.com.ua/ua/novus_ns5502_grey/p23252575/', 'https://rozetka.com.ua/ua/sigma_comfort_50_light_ds_rd/p10603422/', 'https://rozetka.com.ua/ua/philips_xenium_cte106rd/p32591863/', 'https://rozetka.com.ua/ua/tplink_neffos_tp902a26ua_gr/p20431879/', 'https://rozetka.com.ua/ua/alcatel_1054d_3aalua1/p24458744/', 'https://rozetka.com.ua/ua/doogge_mix_lite_blue/p28173793/', 'https://rozetka.com.ua/ua/viaan_v182_blue/p23575580/', 'https://rozetka.com.ua/ua/leeco_le_pro_3_elite_x722_4_32gb_gold/p27234889/', 'https://rozetka.com.ua/ua/sigma_mobile_comfort_retro_50_red/p29138929/', 'https://rozetka.com.ua/ua/nomi_i242_xtreme_black_green/p11083538/', 'https://rozetka.com.ua/ua/bluboo_s1_4_64gb_white_6970595010316/p25469577/', 'https://rozetka.com.ua/ua/twoe_e240_ds_black/p25103761/', 'https://rozetka.com.ua/ua/meizu_m5s_32gld_eu/p16809944/', 'https://rozetka.com.ua/ua/zte_axon7_grey/p11891839/', 'https://rozetka.com.ua/ua/htc_u11_6gb_128gb_black/p22786704/', 'https://rozetka.com.ua/ua/samsung_galaxy_a3_2017_duos_sm_a320_16gb_gold/p13214901/', 'https://rozetka.com.ua/ua/zte_blade_a610_grey/p11673582/', 'https://rozetka.com.ua/ua/tp_link_neffos_y5l_pearl_white/p10097000/', 'https://rozetka.com.ua/ua/nomi_i241_metal_steel/p7110644/', 'https://rozetka.com.ua/ua/nomi_i177_metal_grey/p7110429/', 'https://rozetka.com.ua/ua/twoe_e180_ds_blgr/p24865121/', 'https://rozetka.com.ua/ua/nomi_i244_black_red/p21812736/', 'https://rozetka.com.ua/ua/zte_blade_a6_lite_black/p25717913/', 'https://rozetka.com.ua/ua/assistant_as_4211/p12532368/', 'https://rozetka.com.ua/ua/lg_g360_titan/p6351213/', 'https://rozetka.com.ua/ua/philips_e106_ds_black/p18505186/', 'https://rozetka.com.ua/ua/apple_iphone_x_256gb_silver/p22726287/', 'https://rozetka.com.ua/ua/twoe_e240_ds_black_708744071132/p33796183/', 'https://rozetka.com.ua/ua/htc_one_x10_ds_slvr/p17049182/', 'https://rozetka.com.ua/ua/sigma_x_treme_pq31_yellow_black/p12550379/', 'https://rozetka.com.ua/ua/blackview_bv6000s_black_roz/p28337281/', 'https://rozetka.com.ua/ua/myphone_tel000395/p33173871/', 'https://rozetka.com.ua/ua/huawei_nova_2_blue/p19133436/', 'https://rozetka.com.ua/ua/zte_blade_v8_mini_black/p25724353/', 'https://rozetka.com.ua/ua/nomi_i184wh_bl/p10682375/', 'https://rozetka.com.ua/ua/prestigio_muze_b7_7511_duo_gold/p22155323/', 'https://rozetka.com.ua/ua/htc_u_ultra_eea_brill_blck/p15246248/', 'https://rozetka.com.ua/ua/sigma_xtreme_io67_green/p32112927/', 'https://rozetka.com.ua/ua/oukitel_u15s_black_euromart/p14532182/', 'https://rozetka.com.ua/ua/blackview_s8_black/p28902865/', 'https://rozetka.com.ua/ua/bravis_c241_brace/p32907943/', 'https://rozetka.com.ua/ua/nous_ns_5002_grey/p22386967/', 'https://rozetka.com.ua/ua/sigma_comfort_50_light_ds_gr/p10603401/', 'https://rozetka.com.ua/ua/zte_blade_a910_gold/p11888472/', 'https://rozetka.com.ua/ua/prestigio_pfp1285duoblack/p35894318/', 'https://rozetka.com.ua/ua/samsung_galaxy_s8_plus_128gb_coral_blue/p23768400/', 'https://rozetka.com.ua/ua/elari_lr_npc_slv/p17635104/', 'https://rozetka.com.ua/ua/doogee_s30_gold_roz/p28300513/', 'https://rozetka.com.ua/ua/apple_iphone_7_plus_128gb_black/p11252665/', 'https://rozetka.com.ua/ua/leeco_le_s3_x626_4_64gb_gld_int/p27231257/', 'https://rozetka.com.ua/ua/motorola_g5s_xt1794_bg/p21840505/', 'https://rozetka.com.ua/ua/prestigio_pfp1242duoblack/p24378543/', 'https://rozetka.com.ua/ua/xiaomi_redmi_5_plus_4_64_blue/p30404879/', 'https://rozetka.com.ua/ua/zte_blade_l5_grey/p11874752/', 'https://rozetka.com.ua/ua/zte_blade_l5_plus_gold/p11876418/', 'https://rozetka.com.ua/ua/assistant_as_5411_dark_blue/p13290363/', 'https://rozetka.com.ua/ua/assistant_as_502_shot_dark_blue/p32105975/', 'https://rozetka.com.ua/ua/nomi_i245_xtreme_black_blue/p21812505/', 'https://rozetka.com.ua/ua/philips_s318_dual_sim_grey/p27925385/', 'https://rozetka.com.ua/ua/maxcom_mm141_gray/p30963343/', 'https://rozetka.com.ua/ua/lg_lgh870s_acisbk/p27027321/', 'https://rozetka.com.ua/ua/zte_blade_v8_lite_blue/p25723257/', 'https://rozetka.com.ua/ua/asus_zenfone_4_max_90ax00h2_m02220/p35930319/', 'https://rozetka.com.ua/ua/tp_link_neffos_y5s_grey/p35089279/', 'https://rozetka.com.ua/ua/motorola_pa5d0075ua/p13744838/', 'https://rozetka.com.ua/ua/sigma_mobile_comfort_50_senior_white/p20342595/', 'https://rozetka.com.ua/ua/asus_zenfone_live_zb553kl_5g088ww/p28562233/', 'https://rozetka.com.ua/ua/bravis_a503_joy_gold/p12141396/', 'https://rozetka.com.ua/ua/motorola_pa6j0041ua/p18843076/', 'https://rozetka.com.ua/ua/sigma_mobile_xstyle_28_flip_gold_mocca/p22033705/', 'https://rozetka.com.ua/ua/tp_link_neffos_y5_dark_grey/p12828559/', 'https://rozetka.com.ua/ua/twoe_e450r_ds_gray/p30263719/', 'https://rozetka.com.ua/ua/lg_lgm250_acisgk/p15215245/', 'https://rozetka.com.ua/ua/zte_blade_a520_gold/p25699945/', 'https://rozetka.com.ua/ua/samsung_galaxy_s8_plus_64gb_gold/p15260346/', 'https://rozetka.com.ua/ua/lg_lgh870ds_acisbk/p15213614/', 'https://rozetka.com.ua/ua/sony_xperia_xa1_dual_gold/p14664104/', 'https://rozetka.com.ua/ua/astro_a180_rx_black_camo/p9235237/', 'https://rozetka.com.ua/ua/twoe_r240_ds_black/p24871211/', 'https://rozetka.com.ua/ua/blackview_a10_barley_golden/p28902713/', 'https://rozetka.com.ua/ua/philips_xenium_cte331br/p32591327/', 'https://rozetka.com.ua/ua/sigma_xtreme_dt68_black_red_4827798337721/p35572012/', 'https://rozetka.com.ua/ua/sigma_comfort_4827798337516/p35572579/', 'https://rozetka.com.ua/ua/twoe_e181_ds_red/p33796167/', 'https://rozetka.com.ua/ua/twoe_e500a_ds_black/p27169473/', 'https://rozetka.com.ua/ua/myphone_tel000418/p33232151/', 'https://rozetka.com.ua/ua/sony_xperia_l2_pink/p31697231/', 'https://rozetka.com.ua/ua/apple_iphone_8_256gb_silver/p22726189/', 'https://rozetka.com.ua/ua/tp_link_neffos_c5_dark_grey/p8688779/', 'https://rozetka.com.ua/ua/htc_u11_4gb_64gb_silver/p19017887/', 'https://rozetka.com.ua/ua/apple_iphone_7_128gb_rose_gold/p11251293/', 'https://rozetka.com.ua/ua/alcatel_5046d_2dalua1/p24498894/', 'https://rozetka.com.ua/ua/archos_55b_8gb_platinum/p17000528/', 'https://rozetka.com.ua/ua/motorola_pa800126ua/p18850265/', 'https://rozetka.com.ua/ua/maxcom_mm818_black/p30963423/', 'https://rozetka.com.ua/ua/viaan_v241_black/p11704270/', 'https://rozetka.com.ua/ua/viaan_t101_sb/p18148020/', 'https://rozetka.com.ua/ua/sony_xperia_z5_dual_white/p4954130/', 'https://rozetka.com.ua/ua/sony_xa2_ultra_gold/p34328943/', 'https://rozetka.com.ua/ua/prestigio_t3psp5510duoblack/p35036471/', 'https://rozetka.com.ua/ua/zte_blade_l110_yellow/p11874514/', 'https://rozetka.com.ua/ua/china_mobile_a3s_2_16gb_silver_m653s/p32943167/', 'https://rozetka.com.ua/ua/sony_xa2_black/p34482935/', 'https://rozetka.com.ua/ua/sigma_x_treme_pq34_black/p35607544/', 'https://rozetka.com.ua/ua/huawei_y5_ii_black/p9659850/', 'https://rozetka.com.ua/ua/nomi_i5071_iron_x1_black_330083/p34521159/', 'https://rozetka.com.ua/ua/asus_zenfone_4_max_90ax00h2_m02200/p34993415/', 'https://rozetka.com.ua/ua/asus_zenfone_go_zb500kg_3g007ww/p35034919/', 'https://rozetka.com.ua/ua/sigma_mobile_x_style_33_steel_red/p32111047/', 'https://rozetka.com.ua/ua/nomi_i5511_space_m1_black/p33493423/', 'https://rozetka.com.ua/ua/33551119/p33551119/', 'https://rozetka.com.ua/ua/archos_core_50_lite_503558/p33813639/', 'https://rozetka.com.ua/ua/viaan_v281a_blue/p34974671/', 'https://rozetka.com.ua/ua/35160135/p35160135/', 'https://rozetka.com.ua/ua/nokia_1_11frtr01a06/p35468523/', 'https://rozetka.com.ua/ua/nokia_6_new_11pl2b01a11/p35468880/', 'https://rozetka.com.ua/ua/35551223/p35551223/', 'https://rozetka.com.ua/ua/35551231/p35551231/', 'https://rozetka.com.ua/ua/35551295/p35551295/', 'https://rozetka.com.ua/ua/htc_desire_10pro_64gb_blk_mel/p16949318/', 'https://rozetka.com.ua/ua/motorola_g5s_plus_xt1805_bg/p21847617/', 'https://rozetka.com.ua/ua/doogge_x10b/p23760894/', 'https://rozetka.com.ua/ua/25602153/p25602153/', 'https://rozetka.com.ua/ua/motorola_moto_x4_sterling_blue/p27170657/', 'https://rozetka.com.ua/ua/33550767/p33550767/', 'https://rozetka.com.ua/ua/34523119/p34523119/', 'https://rozetka.com.ua/ua/34985423/p34985423/', 'https://rozetka.com.ua/ua/34985495/p34985495/', 'https://rozetka.com.ua/ua/35160159/p35160159/', 'https://rozetka.com.ua/ua/35551119/p35551119/', 'https://rozetka.com.ua/ua/35551127/p35551127/', 'https://rozetka.com.ua/ua/35551135/p35551135/', 'https://rozetka.com.ua/ua/35551143/p35551143/', 'https://rozetka.com.ua/ua/35551151/p35551151/', 'https://rozetka.com.ua/ua/35551159/p35551159/', 'https://rozetka.com.ua/ua/35551175/p35551175/', 'https://rozetka.com.ua/ua/35551183/p35551183/', 'https://rozetka.com.ua/ua/35551191/p35551191/', 'https://rozetka.com.ua/ua/35551199/p35551199/', 'https://rozetka.com.ua/ua/35551207/p35551207/', 'https://rozetka.com.ua/ua/35551215/p35551215/', 'https://rozetka.com.ua/ua/35551239/p35551239/', 'https://rozetka.com.ua/ua/35551247/p35551247/', 'https://rozetka.com.ua/ua/35551255/p35551255/', 'https://rozetka.com.ua/ua/35551263/p35551263/', 'https://rozetka.com.ua/ua/35551279/p35551279/', 'https://rozetka.com.ua/ua/35551287/p35551287/', 'https://rozetka.com.ua/ua/35551303/p35551303/', 'https://rozetka.com.ua/ua/35577269/p35577269/', 'https://rozetka.com.ua/ua/35577409/p35577409/', 'https://rozetka.com.ua/ua/35577486/p35577486/', 'https://rozetka.com.ua/ua/35577549/p35577549/', 'https://rozetka.com.ua/ua/35577696/p35577696/', 'https://rozetka.com.ua/ua/35577759/p35577759/', 'https://rozetka.com.ua/ua/35578074/p35578074/', 'https://rozetka.com.ua/ua/doogee_x60l_black/p35916144/', 'https://rozetka.com.ua/ua/zte_blade_a510_blue/p11879617/', 'https://rozetka.com.ua/ua/zte_blade_a601_gold/p11881997/', 'https://rozetka.com.ua/ua/zte_blade_x3_gold/p11884048/', 'https://rozetka.com.ua/ua/apple_iphone_6s_32gb_space_gray/p11916969/', 'https://rozetka.com.ua/ua/htc_u_ultra_eea_ice_wht/p15249118/', 'https://rozetka.com.ua/ua/huawei_y3_2017_ds_gld/p17723178/', 'https://rozetka.com.ua/ua/28782153/p28782153/', 'https://rozetka.com.ua/ua/fly_fs527_nimbus_17_gold/p30261191/', 'https://rozetka.com.ua/ua/xiaomi_redmi_5_2_16_gold/p30392935/', 'https://rozetka.com.ua/ua/samsung_sm_j250fzddsek/p31385423/', 'https://rozetka.com.ua/ua/32125207/p32125207/', 'https://rozetka.com.ua/ua/33550759/p33550759/', 'https://rozetka.com.ua/ua/33550775/p33550775/', 'https://rozetka.com.ua/ua/33550887/p33550887/', 'https://rozetka.com.ua/ua/33550967/p33550967/', 'https://rozetka.com.ua/ua/33763247/p33763247/', 'https://rozetka.com.ua/ua/34522991/p34522991/', 'https://rozetka.com.ua/ua/34523007/p34523007/', 'https://rozetka.com.ua/ua/34523055/p34523055/', 'https://rozetka.com.ua/ua/34523159/p34523159/', 'https://rozetka.com.ua/ua/34523191/p34523191/', 'https://rozetka.com.ua/ua/34523223/p34523223/', 'https://rozetka.com.ua/ua/34523303/p34523303/', 'https://rozetka.com.ua/ua/34985455/p34985455/', 'https://rozetka.com.ua/ua/34985823/p34985823/', 'https://rozetka.com.ua/ua/34985935/p34985935/', 'https://rozetka.com.ua/ua/34986007/p34986007/', 'https://rozetka.com.ua/ua/34986031/p34986031/', 'https://rozetka.com.ua/ua/34986047/p34986047/', 'https://rozetka.com.ua/ua/34986079/p34986079/', 'https://rozetka.com.ua/ua/34986135/p34986135/', 'https://rozetka.com.ua/ua/maxcom_mm129_white_green/p35036095/', 'https://rozetka.com.ua/ua/35160247/p35160247/', 'https://rozetka.com.ua/ua/35160367/p35160367/', 'https://rozetka.com.ua/ua/35160391/p35160391/', 'https://rozetka.com.ua/ua/35160567/p35160567/', 'https://rozetka.com.ua/ua/35160655/p35160655/', 'https://rozetka.com.ua/ua/alcatel_onetouch6044d_black/p36191360/', 'https://rozetka.com.ua/ua/archos_55b_16gb_platinum/p17000654/', 'https://rozetka.com.ua/ua/24241721/p24241721/', 'https://rozetka.com.ua/ua/27161601/p27161601/', 'https://rozetka.com.ua/ua/astro_a188_navy/p27164937/', 'https://rozetka.com.ua/ua/astro_a242_red/p27164961/', 'https://rozetka.com.ua/ua/lephone_k1/p32523487/', 'https://rozetka.com.ua/ua/33763063/p33763063/', 'https://rozetka.com.ua/ua/33763335/p33763335/', 'https://rozetka.com.ua/ua/33763367/p33763367/', 'https://rozetka.com.ua/ua/34523103/p34523103/', 'https://rozetka.com.ua/ua/34523167/p34523167/', 'https://rozetka.com.ua/ua/34985511/p34985511/', 'https://rozetka.com.ua/ua/maxcom_mm128_black_blue/p35027359/', 'https://rozetka.com.ua/ua/35379335/p35379335/', 'https://rozetka.com.ua/ua/35379367/p35379367/', 'https://rozetka.com.ua/ua/35379609/p35379609/', 'https://rozetka.com.ua/ua/35379763/p35379763/', 'https://rozetka.com.ua/ua/35379826/p35379826/', 'https://rozetka.com.ua/ua/35380029/p35380029/', 'https://rozetka.com.ua/ua/35380057/p35380057/', 'https://rozetka.com.ua/ua/35380078/p35380078/', 'https://rozetka.com.ua/ua/35380099/p35380099/', 'https://rozetka.com.ua/ua/35380141/p35380141/', 'https://rozetka.com.ua/ua/35380204/p35380204/', 'https://rozetka.com.ua/ua/35380267/p35380267/', 'https://rozetka.com.ua/ua/35380330/p35380330/', 'https://rozetka.com.ua/ua/35440719/p35440719/', 'https://rozetka.com.ua/ua/35440747/p35440747/', 'https://rozetka.com.ua/ua/35440796/p35440796/', 'https://rozetka.com.ua/ua/35440810/p35440810/', 'https://rozetka.com.ua/ua/35441048/p35441048/', 'https://rozetka.com.ua/ua/35441062/p35441062/', 'https://rozetka.com.ua/ua/35441132/p35441132/', 'https://rozetka.com.ua/ua/35441153/p35441153/', 'https://rozetka.com.ua/ua/35441160/p35441160/', 'https://rozetka.com.ua/ua/35441167/p35441167/', 'https://rozetka.com.ua/ua/35441174/p35441174/', 'https://rozetka.com.ua/ua/35441251/p35441251/', 'https://rozetka.com.ua/ua/35441272/p35441272/', 'https://rozetka.com.ua/ua/35441293/p35441293/', 'https://rozetka.com.ua/ua/35441398/p35441398/', 'https://rozetka.com.ua/ua/35441419/p35441419/', 'https://rozetka.com.ua/ua/35441433/p35441433/', 'https://rozetka.com.ua/ua/35441440/p35441440/', 'https://rozetka.com.ua/ua/35441447/p35441447/', 'https://rozetka.com.ua/ua/35441454/p35441454/', 'https://rozetka.com.ua/ua/35441468/p35441468/', 'https://rozetka.com.ua/ua/35441545/p35441545/', 'https://rozetka.com.ua/ua/35441552/p35441552/', 'https://rozetka.com.ua/ua/35441573/p35441573/', 'https://rozetka.com.ua/ua/35441622/p35441622/', 'https://rozetka.com.ua/ua/35441657/p35441657/', 'https://rozetka.com.ua/ua/35441664/p35441664/', 'https://rozetka.com.ua/ua/35441692/p35441692/', 'https://rozetka.com.ua/ua/35441769/p35441769/', 'https://rozetka.com.ua/ua/35441853/p35441853/', 'https://rozetka.com.ua/ua/35450316/p35450316/', 'https://rozetka.com.ua/ua/35450393/p35450393/', 'https://rozetka.com.ua/ua/35450400/p35450400/', 'https://rozetka.com.ua/ua/35450407/p35450407/', 'https://rozetka.com.ua/ua/35450428/p35450428/', 'https://rozetka.com.ua/ua/35450456/p35450456/', 'https://rozetka.com.ua/ua/35450463/p35450463/', 'https://rozetka.com.ua/ua/35450470/p35450470/', 'https://rozetka.com.ua/ua/35450533/p35450533/', 'https://rozetka.com.ua/ua/35450547/p35450547/', 'https://rozetka.com.ua/ua/35450561/p35450561/', 'https://rozetka.com.ua/ua/35450575/p35450575/', 'https://rozetka.com.ua/ua/35450582/p35450582/', 'https://rozetka.com.ua/ua/35450596/p35450596/', 'https://rozetka.com.ua/ua/35450617/p35450617/', 'https://rozetka.com.ua/ua/35450680/p35450680/', 'https://rozetka.com.ua/ua/35450708/p35450708/', 'https://rozetka.com.ua/ua/doogee_x53_black/p35894794/', 'https://rozetka.com.ua/ua/36325176/p36325176/', 'https://rozetka.com.ua/ua/36325240/p36325240/', 'https://rozetka.com.ua/ua/36325320/p36325320/', 'https://rozetka.com.ua/ua/36325368/p36325368/', 'https://rozetka.com.ua/ua/36325400/p36325400/', 'https://rozetka.com.ua/ua/36325408/p36325408/', 'https://rozetka.com.ua/ua/caterpillar_cat_b25/p287716/', 'https://rozetka.com.ua/ua/alcatel_one_touch_1016d_volcano_black/p7239610/', 'https://rozetka.com.ua/ua/motorola_sm4425ad1u1/p12053966/', 'https://rozetka.com.ua/ua/oukitel_u15prop/p14535212/', 'https://rozetka.com.ua/ua/nokia_6_ds_black/p14653634/', 'https://rozetka.com.ua/ua/17632014/p17632014/', 'https://rozetka.com.ua/ua/24241609/p24241609/', 'https://rozetka.com.ua/ua/24241686/p24241686/', 'https://rozetka.com.ua/ua/24241735/p24241735/', 'https://rozetka.com.ua/ua/24458280/p24458280/', 'https://rozetka.com.ua/ua/24458296/p24458296/', 'https://rozetka.com.ua/ua/24458304/p24458304/', 'https://rozetka.com.ua/ua/27161577/p27161577/', 'https://rozetka.com.ua/ua/27161593/p27161593/', 'https://rozetka.com.ua/ua/prestigio_psp5515duoblack/p27775033/', 'https://rozetka.com.ua/ua/doogee_bl5000_blue/p28305969/', 'https://rozetka.com.ua/ua/28343969/p28343969/', 'https://rozetka.com.ua/ua/28481393/p28481393/', 'https://rozetka.com.ua/ua/28481441/p28481441/', 'https://rozetka.com.ua/ua/32125191/p32125191/', 'https://rozetka.com.ua/ua/lephone_k7/p32521519/', 'https://rozetka.com.ua/ua/32532167/p32532167/', 'https://rozetka.com.ua/ua/32886287/p32886287/', 'https://rozetka.com.ua/ua/32886295/p32886295/', 'https://rozetka.com.ua/ua/32907007/p32907007/', 'https://rozetka.com.ua/ua/32907055/p32907055/', 'https://rozetka.com.ua/ua/32907071/p32907071/', 'https://rozetka.com.ua/ua/32907087/p32907087/', 'https://rozetka.com.ua/ua/32907111/p32907111/', 'https://rozetka.com.ua/ua/32907127/p32907127/', 'https://rozetka.com.ua/ua/32907143/p32907143/', 'https://rozetka.com.ua/ua/32907167/p32907167/', 'https://rozetka.com.ua/ua/32907183/p32907183/', 'https://rozetka.com.ua/ua/myphone_tel000371/p33173847/', 'https://rozetka.com.ua/ua/33550719/p33550719/', 'https://rozetka.com.ua/ua/33550727/p33550727/', 'https://rozetka.com.ua/ua/33550855/p33550855/', 'https://rozetka.com.ua/ua/33551031/p33551031/', 'https://rozetka.com.ua/ua/33551047/p33551047/', 'https://rozetka.com.ua/ua/33551063/p33551063/', 'https://rozetka.com.ua/ua/33551087/p33551087/', 'https://rozetka.com.ua/ua/33763055/p33763055/', 'https://rozetka.com.ua/ua/33763071/p33763071/', 'https://rozetka.com.ua/ua/33763079/p33763079/', 'https://rozetka.com.ua/ua/33763087/p33763087/', 'https://rozetka.com.ua/ua/33763095/p33763095/', 'https://rozetka.com.ua/ua/33763103/p33763103/', 'https://rozetka.com.ua/ua/33763119/p33763119/', 'https://rozetka.com.ua/ua/33763127/p33763127/', 'https://rozetka.com.ua/ua/33763151/p33763151/', 'https://rozetka.com.ua/ua/33763159/p33763159/', 'https://rozetka.com.ua/ua/33763191/p33763191/', 'https://rozetka.com.ua/ua/33763255/p33763255/', 'https://rozetka.com.ua/ua/33763327/p33763327/', 'https://rozetka.com.ua/ua/33763343/p33763343/', 'https://rozetka.com.ua/ua/33763351/p33763351/', 'https://rozetka.com.ua/ua/33763383/p33763383/', 'https://rozetka.com.ua/ua/archos_core_50_503585/p33808079/', 'https://rozetka.com.ua/ua/archos_sense_50dc_503525/p33825463/', 'https://rozetka.com.ua/ua/archos_sense_55dc_503524/p33921431/', 'https://rozetka.com.ua/ua/archos_sense_55s_503568/p33922279/', 'https://rozetka.com.ua/ua/34522999/p34522999/', 'https://rozetka.com.ua/ua/34523031/p34523031/', 'https://rozetka.com.ua/ua/34523047/p34523047/', 'https://rozetka.com.ua/ua/34523079/p34523079/', 'https://rozetka.com.ua/ua/34523087/p34523087/', 'https://rozetka.com.ua/ua/34523095/p34523095/', 'https://rozetka.com.ua/ua/34523151/p34523151/', 'https://rozetka.com.ua/ua/34523183/p34523183/', 'https://rozetka.com.ua/ua/34523207/p34523207/', 'https://rozetka.com.ua/ua/34523239/p34523239/', 'https://rozetka.com.ua/ua/34523255/p34523255/', 'https://rozetka.com.ua/ua/34523263/p34523263/', 'https://rozetka.com.ua/ua/34523279/p34523279/', 'https://rozetka.com.ua/ua/34523295/p34523295/', 'https://rozetka.com.ua/ua/34977767/p34977767/', 'https://rozetka.com.ua/ua/34985431/p34985431/', 'https://rozetka.com.ua/ua/34985551/p34985551/', 'https://rozetka.com.ua/ua/34985663/p34985663/', 'https://rozetka.com.ua/ua/34985727/p34985727/', 'https://rozetka.com.ua/ua/34985783/p34985783/', 'https://rozetka.com.ua/ua/34985879/p34985879/', 'https://rozetka.com.ua/ua/34985967/p34985967/', 'https://rozetka.com.ua/ua/34985983/p34985983/', 'https://rozetka.com.ua/ua/34986063/p34986063/', 'https://rozetka.com.ua/ua/34986119/p34986119/', 'https://rozetka.com.ua/ua/34986175/p34986175/', 'https://rozetka.com.ua/ua/35006783/p35006783/', 'https://rozetka.com.ua/ua/35006847/p35006847/', 'https://rozetka.com.ua/ua/35006895/p35006895/', 'https://rozetka.com.ua/ua/35007151/p35007151/', 'https://rozetka.com.ua/ua/35007247/p35007247/', 'https://rozetka.com.ua/ua/35007351/p35007351/', 'https://rozetka.com.ua/ua/35007375/p35007375/', 'https://rozetka.com.ua/ua/35007423/p35007423/', 'https://rozetka.com.ua/ua/35160191/p35160191/', 'https://rozetka.com.ua/ua/35160263/p35160263/', 'https://rozetka.com.ua/ua/35160287/p35160287/', 'https://rozetka.com.ua/ua/35160327/p35160327/', 'https://rozetka.com.ua/ua/35160423/p35160423/', 'https://rozetka.com.ua/ua/35160463/p35160463/', 'https://rozetka.com.ua/ua/35160551/p35160551/', 'https://rozetka.com.ua/ua/35160615/p35160615/', 'https://rozetka.com.ua/ua/35160687/p35160687/', 'https://rozetka.com.ua/ua/35577934/p35577934/', 'https://rozetka.com.ua/ua/35578011/p35578011/', 'https://rozetka.com.ua/ua/astro_a184_navy/p36829744/', 'https://rozetka.com.ua/ua/astro_a185_navy/p36863104/', 'https://rozetka.com.ua/ua/huawei_p20_4_128gb_gold/p36387504/', 'https://rozetka.com.ua/ua/huawei_p20_pro_6_128gb_twilight/p36387736/', 'https://rozetka.com.ua/ua/nokia_7_plus_11b2nb01a01/p35493231/', 'https://rozetka.com.ua/ua/sigma_mobile_4827798373712/p36517176/', 'https://rozetka.com.ua/ua/sigma_mobile_4827798337417/p36548568/', 'https://rozetka.com.ua/ua/huawei_y5_ii_gold_lifecell/p12657227/', 'https://rozetka.com.ua/ua/apple_iphone_8_plus_64gb_silver/p22726259/', 'https://rozetka.com.ua/ua/alcatel_one_touch_idol_3_6045y_dark_grey/p8916328/', 'https://rozetka.com.ua/ua/xiaomi_redmi_4x_32_black_roz/p22437607/', 'https://rozetka.com.ua/ua/xiaomi_mi6_6_128_blue_roz/p23508467/', 'https://rozetka.com.ua/ua/doogee_x30_gold/p31066039/', 'https://rozetka.com.ua/ua/leeco_le_pro_3_x727_4_64gb_snapdragon_821_gold_world/p25441809/', 'https://rozetka.com.ua/ua/motorola_g5_xt1685_gry/p17706834/', 'https://rozetka.com.ua/ua/apple_iphone_8_64gb_gold/p22726217/', 'https://rozetka.com.ua/ua/apple_iphone_8_plus_256gb_gold/p22726245/', 'https://rozetka.com.ua/ua/34986151/p34986151/', 'https://rozetka.com.ua/ua/16842872/p16842872/', 'https://rozetka.com.ua/ua/16845740/p16845740/', 'https://rozetka.com.ua/ua/16847210/p16847210/', 'https://rozetka.com.ua/ua/16847456/p16847456/', 'https://rozetka.com.ua/ua/16847990/p16847990/', 'https://rozetka.com.ua/ua/astro_a228_red/p21694478/', 'https://rozetka.com.ua/ua/xiaomi_redmi_note4_4_64gb_grey/p23503427/', 'https://rozetka.com.ua/ua/doogee_y6cmblue/p24783031/', 'https://rozetka.com.ua/ua/doogee_x20_black/p24836281/', 'https://rozetka.com.ua/ua/doogee_mix_4_64gb_black/p28303569/', 'https://rozetka.com.ua/ua/blackview_bv4000_black/p28388457/', 'https://rozetka.com.ua/ua/xiaomi_redmi_5_plus_3_32_gold/p30404823/', 'https://rozetka.com.ua/ua/33763239/p33763239/', 'https://rozetka.com.ua/ua/sigma_mobile_x_style_17_up_black/p33805591/', 'https://rozetka.com.ua/ua/archos_55_platinum_503107/p33924583/', 'https://rozetka.com.ua/ua/doogge_x5_max_black_roz/p35100063/', 'https://rozetka.com.ua/ua/35551167/p35551167/', 'https://rozetka.com.ua/ua/35577332/p35577332/', 'https://rozetka.com.ua/ua/35577612/p35577612/', 'https://rozetka.com.ua/ua/35577822/p35577822/', 'https://rozetka.com.ua/ua/35577885/p35577885/', 'https://rozetka.com.ua/ua/sony_xperia_xz2_compact_silver/p35622629/', 'https://rozetka.com.ua/ua/asus_zenfone_90ax0171_m01200/p35931551/', 'https://rozetka.com.ua/ua/astro_b181_black_orange/p6569740/', 'https://rozetka.com.ua/ua/sony_xperia_xa_dual_lime_gold/p9308177/', 'https://rozetka.com.ua/ua/huawei_y3_ii_black/p10729415/', 'https://rozetka.com.ua/ua/oukitel_k6000_pro_wh_eu/p12146310/', 'https://rozetka.com.ua/ua/prestigio_grace_z5_5530_duo_black/p12483123/', 'https://rozetka.com.ua/ua/prestigio_grace_r7_7501_duo_gold/p12483606/', 'https://rozetka.com.ua/ua/xiaomi_redmi_note_5a_gray_eu/p24231242/', 'https://rozetka.com.ua/ua/34523127/p34523127/', 'https://rozetka.com.ua/ua/34523287/p34523287/', 'https://rozetka.com.ua/ua/34985847/p34985847/', 'https://rozetka.com.ua/ua/35160631/p35160631/', 'https://rozetka.com.ua/ua/sigma_mobile_4827798337219/p36700672/', 'https://rozetka.com.ua/ua/doogge_x5_black/p7362922/', 'https://rozetka.com.ua/ua/assistant_as_6431_rider_black/p9983866/', 'https://rozetka.com.ua/ua/meizu_m3s_16_gd_eu/p10658505/', 'https://rozetka.com.ua/ua/huawei_novagold/p12424113/', 'https://rozetka.com.ua/ua/xiaomi_redmi_4a_p29804/p17500822/', 'https://rozetka.com.ua/ua/xiaomi_mi6_664_blk/p17639196/', 'https://rozetka.com.ua/ua/motorola_pa700043ua/p18853345/', 'https://rozetka.com.ua/ua/apple_fkqn2rm_a/p21638926/', 'https://rozetka.com.ua/ua/ergo_f182_point/p25040521/', 'https://rozetka.com.ua/ua/ergo_f242_turbo/p25050473/', 'https://rozetka.com.ua/ua/zte_blade_a6_black/p25720777/', 'https://rozetka.com.ua/ua/33763199/p33763199/', 'https://rozetka.com.ua/ua/34522983/p34522983/', 'https://rozetka.com.ua/ua/34523023/p34523023/', 'https://rozetka.com.ua/ua/35380127/p35380127/', 'https://rozetka.com.ua/ua/samsung_galaxy_j5_ds_black/p3818036/', 'https://rozetka.com.ua/ua/samsung_galaxy_j7_ds_black/p3818722/', 'https://rozetka.com.ua/ua/apple_iphone_5s_16gb_space_gray_me432rk_a/p6800444/', 'https://rozetka.com.ua/ua/globex_mint_red/p6956558/', 'https://rozetka.com.ua/ua/microsoft_lumia_650_dual_sim_white/p7245166/', 'https://rozetka.com.ua/ua/tp_link_neffos_c5_max_dark_grey/p8711340/', 'https://rozetka.com.ua/ua/lgh845_acisgd/p9154520/', 'https://rozetka.com.ua/ua/doogge_x5_max_white/p9195743/', 'https://rozetka.com.ua/ua/astro_a240_white/p9271217/', 'https://rozetka.com.ua/ua/lenovo_vibe_p1_pro_silver/p9651793/', 'https://rozetka.com.ua/ua/lenovo_c2_k10a40_black/p9855626/', 'https://rozetka.com.ua/ua/motorola_sm4372ad1k7/p9859602/', 'https://rozetka.com.ua/ua/meizu_m3_note_16_gr_eu/p9859672/', 'https://rozetka.com.ua/ua/prestigio_multiphone_3403_duo_green/p9871033/', 'https://rozetka.com.ua/ua/fly_ezzy_8_grey/p9967010/', 'https://rozetka.com.ua/ua/ergo_f281_link_bk/p10225541/', 'https://rozetka.com.ua/ua/pixus_hit/p10698888/', 'https://rozetka.com.ua/ua/fly_fs454_nimbus_8_black/p11030520/', 'https://rozetka.com.ua/ua/fly_fs505_nimbus_7_black/p11030856/', 'https://rozetka.com.ua/ua/apple_iphone_7_256gb_jet_black/p11256522/', 'https://rozetka.com.ua/ua/sony_xperia_x_compact_blue/p11850868/', 'https://rozetka.com.ua/ua/zte_blade_v7_lite_dark_grey/p11884069/', 'https://rozetka.com.ua/ua/zte_blade_v7_gold/p11888262/', 'https://rozetka.com.ua/ua/motorola_sm4389ad1u1/p12045314/', 'https://rozetka.com.ua/ua/lenovo_pa4n0007ua/p12087055/', 'https://rozetka.com.ua/ua/xiaomi_mi4_3_16gb_white/p12151455/', 'https://rozetka.com.ua/ua/alcatel_4034d_volcano_black/p12212642/', 'https://rozetka.com.ua/ua/asus_zenfone_3_ze520kl_white/p12291364/', 'https://rozetka.com.ua/ua/meizu_m3_note_32_gr_eu/p12383324/', 'https://rozetka.com.ua/ua/doogee_y100x_nova_white/p12557442/', 'https://rozetka.com.ua/ua/doogee_y100_plus_valencia2_gr/p12566829/', 'https://rozetka.com.ua/ua/doogge_y300_gold_r/p12569552/', 'https://rozetka.com.ua/ua/doogee_y200/p12569643/', 'https://rozetka.com.ua/ua/pixus_jet_gold/p12635975/', 'https://rozetka.com.ua/ua/meizu_m3e_32_gr_eu/p12721753/', 'https://rozetka.com.ua/ua/letv_leeco_cool1_32gb_ds_sl_mel/p12739050/', 'https://rozetka.com.ua/ua/letv_le1s_32gb_ds_sl_mel/p12739057/', 'https://rozetka.com.ua/ua/letv_x820max_2_32gb_ds_rg/p12739064/', 'https://rozetka.com.ua/ua/letv_le1s_16gb_ds_gd_mel/p12739071/', 'https://rozetka.com.ua/ua/letv_max_900_64gb_ds_sl_mel/p12739078/', 'https://rozetka.com.ua/ua/alcatel_5051d_metal_silver/p12838849/', 'https://rozetka.com.ua/ua/alcatel_8050d_metallic_silver/p12879048/', 'https://rozetka.com.ua/ua/fly_fs509_nimbus_9_black/p13012272/', 'https://rozetka.com.ua/ua/sigma_x_treme_pq35_black/p13012601/', 'https://rozetka.com.ua/ua/impression_imsmart_a503_black/p13119491/', 'https://rozetka.com.ua/ua/ergo_a502_aurum_dual_sim_black/p13686927/', 'https://rozetka.com.ua/ua/oukitel_c5_pro_bk_eu/p13772887/', 'https://rozetka.com.ua/ua/meizu_mx5_16gb_ds_sil_wh_mel/p13993748/', 'https://rozetka.com.ua/ua/lenovo_a560_4gb_whitr_mel/p13993754/', 'https://rozetka.com.ua/ua/meizu_m3_note_2_16_sil_wh_mel/p13993772/', 'https://rozetka.com.ua/ua/doogge_x6_wt_roz/p14294948/', 'https://rozetka.com.ua/ua/meizu_m5_16gb_wht_mel/p14438672/', 'https://rozetka.com.ua/ua/meizu_u20_32gb_wh_mel/p14438690/', 'https://rozetka.com.ua/ua/meizu_m3_note_32gb_gray/p14438744/', 'https://rozetka.com.ua/ua/meizu_u20_16gb_bk_mel/p14438762/', 'https://rozetka.com.ua/ua/meizu_m5_note_32gb_svr_mel/p14438792/', 'https://rozetka.com.ua/ua/prestigio_psp3527duoblack/p14673788/', 'https://rozetka.com.ua/ua/maxwest_nitro4_ds_gld_mel/p14854191/', 'https://rozetka.com.ua/ua/meizu_u10_16gb_wht_mel/p14854226/', 'https://rozetka.com.ua/ua/leeco_le_max_2_4_64gb_gry/p14929940/', 'https://rozetka.com.ua/ua/lenovo_a916_blck_mel/p15061021/', 'https://rozetka.com.ua/ua/ergo_a503_optima_dual_sim_black/p15124791/', 'https://rozetka.com.ua/ua/xiaomi_redmi_4a_2_32_gr_eu/p15218857/', 'https://rozetka.com.ua/ua/15968732/p15968732/', 'https://rozetka.com.ua/ua/15968739/p15968739/', 'https://rozetka.com.ua/ua/15968746/p15968746/', 'https://rozetka.com.ua/ua/15968767/p15968767/', 'https://rozetka.com.ua/ua/15968774/p15968774/', 'https://rozetka.com.ua/ua/15968781/p15968781/', 'https://rozetka.com.ua/ua/16844468/p16844468/', 'https://rozetka.com.ua/ua/16844756/p16844756/', 'https://rozetka.com.ua/ua/16846190/p16846190/', 'https://rozetka.com.ua/ua/16847474/p16847474/', 'https://rozetka.com.ua/ua/16848980/p16848980/', 'https://rozetka.com.ua/ua/umi_diamond_blk_mel/p16949180/', 'https://rozetka.com.ua/ua/16949258/p16949258/', 'https://rozetka.com.ua/ua/oneplus_x_16gb_cer_blk/p16949276/', 'https://rozetka.com.ua/ua/blackview_a8_gld_mel/p16949294/', 'https://rozetka.com.ua/ua/lg_x_view_bk_mel/p16949306/', 'https://rozetka.com.ua/ua/umi_super_blk_mel/p16949324/', 'https://rozetka.com.ua/ua/umi_rome_x_blk_mel/p16949354/', 'https://rozetka.com.ua/ua/umi_diamond_gld_mel/p16949360/', 'https://rozetka.com.ua/ua/umo_london_blk_mel/p16949402/', 'https://rozetka.com.ua/ua/blackview_a8_max_wht_mel/p16949408/', 'https://rozetka.com.ua/ua/lg_v20_h990ds_bk_mel/p16949414/', 'https://rozetka.com.ua/ua/google_pixel_2gb_slvr/p16949438/', 'https://rozetka.com.ua/ua/17088767/p17088767/', 'https://rozetka.com.ua/ua/17088774/p17088774/', 'https://rozetka.com.ua/ua/17088781/p17088781/', 'https://rozetka.com.ua/ua/xiaomi_redmi_note4_3_32gb_gold/p17501137/', 'https://rozetka.com.ua/ua/google_pixel_xl_32gb_blk/p17876124/', 'https://rozetka.com.ua/ua/17918364/p17918364/', 'https://rozetka.com.ua/ua/17918388/p17918388/', 'https://rozetka.com.ua/ua/lg_h791_google_nexus_5x_32gb_quartz/p18098532/', 'https://rozetka.com.ua/ua/google_pixel_xl_128gb_blk/p18098622/', 'https://rozetka.com.ua/ua/lg_h962_v10_ds_64gb_wht/p18098634/', 'https://rozetka.com.ua/ua/doogee_shoot_1_grey/p18817659/', 'https://rozetka.com.ua/ua/prestigio_s7_psp7551duogold/p19405596/', 'https://rozetka.com.ua/ua/philips_cts386/p20761867/', 'https://rozetka.com.ua/ua/lg_h818p_g4_ds_red/p20907951/', 'https://rozetka.com.ua/ua/blackberry_z30_sta100_2_black/p20907979/', 'https://rozetka.com.ua/ua/22222691/p22222691/', 'https://rozetka.com.ua/ua/22222747/p22222747/', 'https://rozetka.com.ua/ua/23596125/p23596125/', 'https://rozetka.com.ua/ua/zte_v5_pro_n939st/p23786478/', 'https://rozetka.com.ua/ua/blackberry_dtek60_32gb_black/p24278268/', 'https://rozetka.com.ua/ua/oneplus_5_a5000_128gb_8gb_ram_grey/p24278324/', 'https://rozetka.com.ua/ua/sharp_fs8010_aquos_s2_64gb_6gb_ram_black/p24278331/', 'https://rozetka.com.ua/ua/blackberry_leap_black/p24278345/', 'https://rozetka.com.ua/ua/24336529/p24336529/', 'https://rozetka.com.ua/ua/24336599/p24336599/', 'https://rozetka.com.ua/ua/24336669/p24336669/', 'https://rozetka.com.ua/ua/24336970/p24336970/', 'https://rozetka.com.ua/ua/24337054/p24337054/', 'https://rozetka.com.ua/ua/24337215/p24337215/', 'https://rozetka.com.ua/ua/24337250/p24337250/', 'https://rozetka.com.ua/ua/24337460/p24337460/', 'https://rozetka.com.ua/ua/24337516/p24337516/', 'https://rozetka.com.ua/ua/24337747/p24337747/', 'https://rozetka.com.ua/ua/24337957/p24337957/', 'https://rozetka.com.ua/ua/24338048/p24338048/', 'https://rozetka.com.ua/ua/philips_x586_dual_sim_black/p24375932/', 'https://rozetka.com.ua/ua/philips_xenium_e311_ds_dark_blue/p24376562/', 'https://rozetka.com.ua/ua/apple_iphone_se_128gb_rose_gold/p24386208/', 'https://rozetka.com.ua/ua/24458552/p24458552/', 'https://rozetka.com.ua/ua/24458560/p24458560/', 'https://rozetka.com.ua/ua/24458568/p24458568/', 'https://rozetka.com.ua/ua/24458592/p24458592/', 'https://rozetka.com.ua/ua/24458600/p24458600/', 'https://rozetka.com.ua/ua/24458632/p24458632/', 'https://rozetka.com.ua/ua/24458648/p24458648/', 'https://rozetka.com.ua/ua/24458656/p24458656/', 'https://rozetka.com.ua/ua/24458664/p24458664/', 'https://rozetka.com.ua/ua/24458688/p24458688/', 'https://rozetka.com.ua/ua/24458720/p24458720/', 'https://rozetka.com.ua/ua/bravis_x500_trace/p25040449/', 'https://rozetka.com.ua/ua/25603401/p25603401/', 'https://rozetka.com.ua/ua/25766545/p25766545/', 'https://rozetka.com.ua/ua/25766553/p25766553/', 'https://rozetka.com.ua/ua/26163905/p26163905/', 'https://rozetka.com.ua/ua/26164241/p26164241/', 'https://rozetka.com.ua/ua/26164361/p26164361/', 'https://rozetka.com.ua/ua/sigma_xtreme_pq27_plus_4827798374870/p27752305/', 'https://rozetka.com.ua/ua/28277081/p28277081/', 'https://rozetka.com.ua/ua/28277121/p28277121/', 'https://rozetka.com.ua/ua/28277161/p28277161/', 'https://rozetka.com.ua/ua/28277193/p28277193/', 'https://rozetka.com.ua/ua/28277233/p28277233/', 'https://rozetka.com.ua/ua/28277297/p28277297/', 'https://rozetka.com.ua/ua/28277321/p28277321/', 'https://rozetka.com.ua/ua/28277361/p28277361/', 'https://rozetka.com.ua/ua/28277401/p28277401/', 'https://rozetka.com.ua/ua/28277457/p28277457/', 'https://rozetka.com.ua/ua/doogee_x20_2_16gb_gold_roz/p28288305/', 'https://rozetka.com.ua/ua/28343977/p28343977/', 'https://rozetka.com.ua/ua/28343993/p28343993/', 'https://rozetka.com.ua/ua/28344001/p28344001/', 'https://rozetka.com.ua/ua/28344025/p28344025/', 'https://rozetka.com.ua/ua/28344033/p28344033/', 'https://rozetka.com.ua/ua/blackview_p2_lite_black/p28376937/', 'https://rozetka.com.ua/ua/28481377/p28481377/', 'https://rozetka.com.ua/ua/archos_50b_cobalt_lite_503386/p31004895/', 'https://rozetka.com.ua/ua/archos_55b_cobalt_lite_503384/p31008759/', 'https://rozetka.com.ua/ua/32140887/p32140887/', 'https://rozetka.com.ua/ua/32140911/p32140911/', 'https://rozetka.com.ua/ua/32140919/p32140919/', 'https://rozetka.com.ua/ua/32140927/p32140927/', 'https://rozetka.com.ua/ua/32140935/p32140935/', 'https://rozetka.com.ua/ua/32140951/p32140951/', 'https://rozetka.com.ua/ua/32140967/p32140967/', 'https://rozetka.com.ua/ua/32140975/p32140975/', 'https://rozetka.com.ua/ua/32140983/p32140983/', 'https://rozetka.com.ua/ua/32140991/p32140991/', 'https://rozetka.com.ua/ua/32141015/p32141015/', 'https://rozetka.com.ua/ua/32141031/p32141031/', 'https://rozetka.com.ua/ua/32141047/p32141047/', 'https://rozetka.com.ua/ua/32141071/p32141071/', 'https://rozetka.com.ua/ua/32141087/p32141087/', 'https://rozetka.com.ua/ua/32141103/p32141103/', 'https://rozetka.com.ua/ua/32141119/p32141119/', 'https://rozetka.com.ua/ua/32141135/p32141135/', 'https://rozetka.com.ua/ua/32141159/p32141159/', 'https://rozetka.com.ua/ua/32886255/p32886255/', 'https://rozetka.com.ua/ua/32886263/p32886263/', 'https://rozetka.com.ua/ua/32886279/p32886279/', 'https://rozetka.com.ua/ua/bravis_c181_senior/p32906327/', 'https://rozetka.com.ua/ua/33763287/p33763287/', 'https://rozetka.com.ua/ua/34523039/p34523039/', 'https://rozetka.com.ua/ua/34523063/p34523063/', 'https://rozetka.com.ua/ua/34523071/p34523071/', 'https://rozetka.com.ua/ua/34523111/p34523111/', 'https://rozetka.com.ua/ua/34523135/p34523135/', 'https://rozetka.com.ua/ua/34523175/p34523175/', 'https://rozetka.com.ua/ua/34523199/p34523199/', 'https://rozetka.com.ua/ua/34523215/p34523215/', 'https://rozetka.com.ua/ua/34523231/p34523231/', 'https://rozetka.com.ua/ua/34523247/p34523247/', 'https://rozetka.com.ua/ua/34523271/p34523271/', 'https://rozetka.com.ua/ua/meizu_m6_note_3_32gb_black_euromart/p34976927/', 'https://rozetka.com.ua/ua/34985439/p34985439/', 'https://rozetka.com.ua/ua/34985471/p34985471/', 'https://rozetka.com.ua/ua/34985599/p34985599/', 'https://rozetka.com.ua/ua/34985639/p34985639/', 'https://rozetka.com.ua/ua/34985695/p34985695/', 'https://rozetka.com.ua/ua/34986095/p34986095/', 'https://rozetka.com.ua/ua/35006831/p35006831/', 'https://rozetka.com.ua/ua/35007191/p35007191/', 'https://rozetka.com.ua/ua/35007335/p35007335/', 'https://rozetka.com.ua/ua/35160103/p35160103/', 'https://rozetka.com.ua/ua/35160223/p35160223/', 'https://rozetka.com.ua/ua/35160511/p35160511/', 'https://rozetka.com.ua/ua/35160591/p35160591/']
print("Scraped {} product pages".format(len(items_links)))

Scraped 900 product pages


Get reviews:

In [5]:
import csv
from langdetect import detect
with open(SCRAPE_FILE, "a") as fl:
    spamwriter = csv.writer(fl)
#     spamwriter.writerow(["stars", "advantage", "drawback", "text"])
    for num, link in enumerate(items_links):
        last_review = None
        if num > 713:
            print("fetching {} out of {} products".format(num, len(items_links)))
            page_c = 1
            while True:
                print("fetching {}comments/page={}".format(link, page_c))
                reviews = get_data(link+"comments/page={}", page_c, '.pp-review-inner')
                if not reviews:
                    break
                if last_review == reviews[-1]:
                    print("END: duplicated content")
                    break
                last_review = reviews[-1]
                for review in reviews:
                    stars = review.find('span', {'class':'g-rating-stars-i'})
                    if stars:
                        texts = review.select(".pp-review-text-i")
                        advantage, drawback = None, None
                        for num, txt in enumerate([i.text.replace("\n", "") for i in texts]):
                            if num == 0:
                                text = txt
                            elif ADVANTANGE_TXT in txt:
                                advantage = txt.replace(ADVANTANGE_TXT, "")
                            elif DRAWBACK_TXT in txt:
                                drawback = txt.replace(DRAWBACK_TXT, "")
                        try:
                            #fetch only ukrainian
                            if detect(text) != 'uk':
                                continue
                        except:
                            continue  
    #                     if len(texts) > 1:
    #                         import ipdb; ipdb.set_trace()
                        spamwriter.writerow([int(stars.attrs['content']), advantage, drawback, text])

                page_c += 1 

fetching 714 out of 900 products
fetching https://rozetka.com.ua/ua/alcatel_4034d_volcano_black/p12212642/comments/page=1
fetching https://rozetka.com.ua/ua/alcatel_4034d_volcano_black/p12212642/comments/page=2
fetching https://rozetka.com.ua/ua/alcatel_4034d_volcano_black/p12212642/comments/page=3
fetching https://rozetka.com.ua/ua/alcatel_4034d_volcano_black/p12212642/comments/page=4
fetching https://rozetka.com.ua/ua/alcatel_4034d_volcano_black/p12212642/comments/page=5
fetching https://rozetka.com.ua/ua/alcatel_4034d_volcano_black/p12212642/comments/page=6
END: duplicated content
fetching 715 out of 900 products
fetching https://rozetka.com.ua/ua/asus_zenfone_3_ze520kl_white/p12291364/comments/page=1
fetching https://rozetka.com.ua/ua/asus_zenfone_3_ze520kl_white/p12291364/comments/page=2
fetching https://rozetka.com.ua/ua/asus_zenfone_3_ze520kl_white/p12291364/comments/page=3
fetching https://rozetka.com.ua/ua/asus_zenfone_3_ze520kl_white/p12291364/comments/page=4
fetching https:/

fetching https://rozetka.com.ua/ua/doogee_y200/p12569643/comments/page=2
fetching https://rozetka.com.ua/ua/doogee_y200/p12569643/comments/page=3
fetching https://rozetka.com.ua/ua/doogee_y200/p12569643/comments/page=4
fetching https://rozetka.com.ua/ua/doogee_y200/p12569643/comments/page=5
END: duplicated content
fetching 721 out of 900 products
fetching https://rozetka.com.ua/ua/pixus_jet_gold/p12635975/comments/page=1
fetching https://rozetka.com.ua/ua/pixus_jet_gold/p12635975/comments/page=2
fetching https://rozetka.com.ua/ua/pixus_jet_gold/p12635975/comments/page=3
fetching https://rozetka.com.ua/ua/pixus_jet_gold/p12635975/comments/page=4
fetching https://rozetka.com.ua/ua/pixus_jet_gold/p12635975/comments/page=5
fetching https://rozetka.com.ua/ua/pixus_jet_gold/p12635975/comments/page=6
fetching https://rozetka.com.ua/ua/pixus_jet_gold/p12635975/comments/page=7
fetching https://rozetka.com.ua/ua/pixus_jet_gold/p12635975/comments/page=8
END: duplicated content
fetching 722 out of

END: duplicated content
fetching 734 out of 900 products
fetching https://rozetka.com.ua/ua/oukitel_c5_pro_bk_eu/p13772887/comments/page=1
fetching https://rozetka.com.ua/ua/oukitel_c5_pro_bk_eu/p13772887/comments/page=2
fetching https://rozetka.com.ua/ua/oukitel_c5_pro_bk_eu/p13772887/comments/page=3
fetching https://rozetka.com.ua/ua/oukitel_c5_pro_bk_eu/p13772887/comments/page=4
fetching https://rozetka.com.ua/ua/oukitel_c5_pro_bk_eu/p13772887/comments/page=5
fetching https://rozetka.com.ua/ua/oukitel_c5_pro_bk_eu/p13772887/comments/page=6
fetching https://rozetka.com.ua/ua/oukitel_c5_pro_bk_eu/p13772887/comments/page=7
END: duplicated content
fetching 735 out of 900 products
fetching https://rozetka.com.ua/ua/meizu_mx5_16gb_ds_sil_wh_mel/p13993748/comments/page=1
fetching https://rozetka.com.ua/ua/meizu_mx5_16gb_ds_sil_wh_mel/p13993748/comments/page=2
fetching https://rozetka.com.ua/ua/meizu_mx5_16gb_ds_sil_wh_mel/p13993748/comments/page=3
END: duplicated content
fetching 736 out o

fetching https://rozetka.com.ua/ua/leeco_le_max_2_4_64gb_gry/p14929940/comments/page=5
END: duplicated content
fetching 748 out of 900 products
fetching https://rozetka.com.ua/ua/lenovo_a916_blck_mel/p15061021/comments/page=1
fetching https://rozetka.com.ua/ua/lenovo_a916_blck_mel/p15061021/comments/page=2
END: duplicated content
fetching 749 out of 900 products
fetching https://rozetka.com.ua/ua/ergo_a503_optima_dual_sim_black/p15124791/comments/page=1
fetching https://rozetka.com.ua/ua/ergo_a503_optima_dual_sim_black/p15124791/comments/page=2
fetching https://rozetka.com.ua/ua/ergo_a503_optima_dual_sim_black/p15124791/comments/page=3
fetching https://rozetka.com.ua/ua/ergo_a503_optima_dual_sim_black/p15124791/comments/page=4
fetching https://rozetka.com.ua/ua/ergo_a503_optima_dual_sim_black/p15124791/comments/page=5
fetching https://rozetka.com.ua/ua/ergo_a503_optima_dual_sim_black/p15124791/comments/page=6
fetching https://rozetka.com.ua/ua/ergo_a503_optima_dual_sim_black/p15124791/

fetching https://rozetka.com.ua/ua/blackview_a8_gld_mel/p16949294/comments/page=16
fetching https://rozetka.com.ua/ua/blackview_a8_gld_mel/p16949294/comments/page=17
END: duplicated content
fetching 766 out of 900 products
fetching https://rozetka.com.ua/ua/lg_x_view_bk_mel/p16949306/comments/page=1
fetching https://rozetka.com.ua/ua/lg_x_view_bk_mel/p16949306/comments/page=2
fetching https://rozetka.com.ua/ua/lg_x_view_bk_mel/p16949306/comments/page=3
fetching https://rozetka.com.ua/ua/lg_x_view_bk_mel/p16949306/comments/page=4
fetching https://rozetka.com.ua/ua/lg_x_view_bk_mel/p16949306/comments/page=5
fetching https://rozetka.com.ua/ua/lg_x_view_bk_mel/p16949306/comments/page=6
fetching https://rozetka.com.ua/ua/lg_x_view_bk_mel/p16949306/comments/page=7
fetching https://rozetka.com.ua/ua/lg_x_view_bk_mel/p16949306/comments/page=8
fetching https://rozetka.com.ua/ua/lg_x_view_bk_mel/p16949306/comments/page=9
fetching https://rozetka.com.ua/ua/lg_x_view_bk_mel/p16949306/comments/page

END: duplicated content
fetching 785 out of 900 products
fetching https://rozetka.com.ua/ua/prestigio_s7_psp7551duogold/p19405596/comments/page=1
fetching https://rozetka.com.ua/ua/prestigio_s7_psp7551duogold/p19405596/comments/page=2
fetching https://rozetka.com.ua/ua/prestigio_s7_psp7551duogold/p19405596/comments/page=3
fetching https://rozetka.com.ua/ua/prestigio_s7_psp7551duogold/p19405596/comments/page=4
fetching https://rozetka.com.ua/ua/prestigio_s7_psp7551duogold/p19405596/comments/page=5
fetching https://rozetka.com.ua/ua/prestigio_s7_psp7551duogold/p19405596/comments/page=6
fetching https://rozetka.com.ua/ua/prestigio_s7_psp7551duogold/p19405596/comments/page=7
fetching https://rozetka.com.ua/ua/prestigio_s7_psp7551duogold/p19405596/comments/page=8
fetching https://rozetka.com.ua/ua/prestigio_s7_psp7551duogold/p19405596/comments/page=9
fetching https://rozetka.com.ua/ua/prestigio_s7_psp7551duogold/p19405596/comments/page=10
fetching https://rozetka.com.ua/ua/prestigio_s7_psp7

fetching https://rozetka.com.ua/ua/24458552/p24458552/comments/page=2
END: duplicated content
fetching 813 out of 900 products
fetching https://rozetka.com.ua/ua/24458560/p24458560/comments/page=1
END: no data:
fetching 814 out of 900 products
fetching https://rozetka.com.ua/ua/24458568/p24458568/comments/page=1
END: no data:
fetching 815 out of 900 products
fetching https://rozetka.com.ua/ua/24458592/p24458592/comments/page=1
fetching https://rozetka.com.ua/ua/24458592/p24458592/comments/page=2
END: duplicated content
fetching 816 out of 900 products
fetching https://rozetka.com.ua/ua/24458600/p24458600/comments/page=1
fetching https://rozetka.com.ua/ua/24458600/p24458600/comments/page=2
fetching https://rozetka.com.ua/ua/24458600/p24458600/comments/page=3
END: duplicated content
fetching 817 out of 900 products
fetching https://rozetka.com.ua/ua/24458632/p24458632/comments/page=1
END: no data:
fetching 818 out of 900 products
fetching https://rozetka.com.ua/ua/24458648/p24458648/comm

END: no data:
fetching 866 out of 900 products
fetching https://rozetka.com.ua/ua/32141103/p32141103/comments/page=1
END: no data:
fetching 867 out of 900 products
fetching https://rozetka.com.ua/ua/32141119/p32141119/comments/page=1
END: no data:
fetching 868 out of 900 products
fetching https://rozetka.com.ua/ua/32141135/p32141135/comments/page=1
END: no data:
fetching 869 out of 900 products
fetching https://rozetka.com.ua/ua/32141159/p32141159/comments/page=1
END: no data:
fetching 870 out of 900 products
fetching https://rozetka.com.ua/ua/32886255/p32886255/comments/page=1
fetching https://rozetka.com.ua/ua/32886255/p32886255/comments/page=2
END: duplicated content
fetching 871 out of 900 products
fetching https://rozetka.com.ua/ua/32886263/p32886263/comments/page=1
fetching https://rozetka.com.ua/ua/32886263/p32886263/comments/page=2
END: duplicated content
fetching 872 out of 900 products
fetching https://rozetka.com.ua/ua/32886279/p32886279/comments/page=1
END: no data:
fetchin